<a href="https://colab.research.google.com/github/jorgecote/Seminario/blob/main/Ejemplo_generaci%C3%B3n_nombres_dinosaurios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generación de nombres de dinosaurios
Desarrollar una red neuronal que pueda generar nombres de dinosaurios utilizando como entrada una letra o varias letras

In [6]:
import numpy as np
np.random.seed(5)
from keras.layers import Input, Dense, SimpleRNN
from keras.models import Model
from keras import backend as K
import tensorflow as tf
import os
import time

#### Carga de dataset
El dataset contiene 1536 nombres diferentes de verdaderos dinosaurios organizados en un archivo de texto plano .txt

In [1]:
from google.colab import drive

drive.mount('/content/drive')   #Se monta el drive para tener disponibles todos los archivos

Mounted at /content/drive


In [3]:
# Se utiliza la función open (Nativa de python) para abrir el archivo y read para leerlos
nombres = open('/content/drive/MyDrive/ejercicios seminario internacional/nombres_dinosaurios.txt','r').read()
nombres = nombres.lower() # Todos los nombres se dejan en minúscula para que no afecten las secuencias de generación


In [4]:
#tam_datos, tam_alfabeto = len(nombres), len(alfabeto)
#nombres=nombres.splitlines()
print(f'El total del texto es: {len(nombres)} caracteres')    # Se calcula el total de caracteres del archivo
print(nombres[:38])                                           # Revisando como luce el conjunto de datos
alfabeto = sorted(set(nombres))                                 #Con la función set python encuentra los caracteres únicos del archivo
print("Total de caracteres: ",len(nombres),", Caracteres únicos: ", len(alfabeto))
print(alfabeto)


El total del texto es: 19909 caracteres
aachenosaurus
aardonyx
abdallahsaurus

Total de caracteres:  19909 , Caracteres únicos:  27
['\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


#### Codificación del dataset
Primero se realiza la codificación asignando un número entero a cada caracter único en el conjunto de datos (archivo)

In [7]:
chars = tf.strings.unicode_split(nombres, input_encoding='UTF-8')   #Se separan los caracteres y se organizan en un tensor
chars

<tf.Tensor: shape=(19909,), dtype=string, numpy=array([b'a', b'a', b'c', ..., b'u', b'u', b'l'], dtype=object)>

In [8]:
car_a_ind = tf.keras.layers.StringLookup(vocabulary=alfabeto, mask_token=None)
#{ car:ind for ind,car in enumerate(sorted(alfabeto))} #Se genera un diccionario asignando un valor único a cada caracter
car_a_ind(chars)
car_a_ind
print(car_a_ind(chars))

tf.Tensor([ 2  2  4 ... 22 22 13], shape=(19909,), dtype=int64)


#### Decodificación del dataset
Se crea una función que ejecute lo contrario, para poder recuperar los caracteres a partir de los índices numéricos:





In [9]:
ind_a_car = tf.keras.layers.StringLookup(vocabulary=car_a_ind.get_vocabulary(), invert=True, mask_token=None)
print(ind_a_car(car_a_ind(chars))[:38])


tf.Tensor(
[b'a' b'a' b'c' b'h' b'e' b'n' b'o' b's' b'a' b'u' b'r' b'u' b's' b'\n'
 b'a' b'a' b'r' b'd' b'o' b'n' b'y' b'x' b'\n' b'a' b'b' b'd' b'a' b'l'
 b'l' b'a' b'h' b's' b'a' b'u' b'r' b'u' b's' b'\n'], shape=(38,), dtype=string)


El conjunto de datos se puede reconstruir de la siguiente manera:

In [10]:
tf.strings.reduce_join(chars, axis=-1).numpy()

b'aachenosaurus\naardonyx\nabdallahsaurus\nabelisaurus\nabrictosaurus\nabrosaurus\nabydosaurus\nacanthopholis\nachelousaurus\nacheroraptor\nachillesaurus\nachillobator\nacristavus\nacrocanthosaurus\nacrotholus\nactiosaurus\nadamantisaurus\nadasaurus\nadelolophus\nadeopapposaurus\naegyptosaurus\naeolosaurus\naepisaurus\naepyornithomimus\naerosteon\naetonyxafromimus\nafrovenator\nagathaumas\naggiosaurus\nagilisaurus\nagnosphitys\nagrosaurus\nagujaceratops\nagustinia\nahshislepelta\nairakoraptor\najancingenia\najkaceratops\nalamosaurus\nalaskacephale\nalbalophosaurus\nalbertaceratops\nalbertadromeus\nalbertavenator\nalbertonykus\nalbertosaurus\nalbinykus\nalbisaurus\nalcovasaurus\nalectrosaurus\naletopelta\nalgoasaurus\nalioramus\naliwalia\nallosaurus\nalmas\nalnashetri\nalocodon\naltirhinus\naltispinax\nalvarezsaurus\nalwalkeria\nalxasaurus\namargasaurus\namargastegos\namargatitanis\namazonsaurus\nammosaurus\nampelosaurus\namphicoelias\namphicoelicaudia\namphisaurus\namtocephale\namtosau

In [16]:
def ind_a_texto(ind):           #Función para recuperar cadena de texto desde índices
  return tf.strings.reduce_join(ind_a_car(ind), axis=-1)

#### Creación de entradas y salidas
Para la creación de las entradas y salidas para el entrenamiento se debe dividir el conjunto con los caracteres desde la posición 0 a tamaño-1 y las salidas desde 1 a tamaño.

Por ejemplo:

Si la cadena es 'Hello'

La entrada sería: 'Hell'

La salida sería: 'ello'


Para esto primero organizamos los datos en un dataset de tensorflow:

In [11]:
all_ids = car_a_ind(tf.strings.unicode_split(nombres, 'UTF-8'))   #Separar los caracteres y asignandoles id único

ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)       #Organizar los índices generados en un dataset de TensorFlow


In [12]:
for ids in ids_dataset.take(14):        #Mostrando los primeros 14 caracteres del dataset
    print(ind_a_car(ids).numpy().decode('utf-8'))

a
a
c
h
e
n
o
s
a
u
r
u
s




Con el conjunto de datos almacenado en el dataset ahora con nel método batch se puede realizar la separación utilizando como argumento el tamaño de la secuencia de entrada

In [13]:
seq_length = 20

In [14]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(2):
  print(ind_a_car(seq))

tf.Tensor(
[b'a' b'a' b'c' b'h' b'e' b'n' b'o' b's' b'a' b'u' b'r' b'u' b's' b'\n'
 b'a' b'a' b'r' b'd' b'o' b'n' b'y'], shape=(21,), dtype=string)
tf.Tensor(
[b'x' b'\n' b'a' b'b' b'd' b'a' b'l' b'l' b'a' b'h' b's' b'a' b'u' b'r'
 b'u' b's' b'\n' b'a' b'b' b'e' b'l'], shape=(21,), dtype=string)


Si se unen de nuevo las secuencias lo que se obtiene es:

In [17]:
for seq in sequences.take(5):
  print(ind_a_texto(seq).numpy())

b'aachenosaurus\naardony'
b'x\nabdallahsaurus\nabel'
b'isaurus\nabrictosaurus'
b'\nabrosaurus\nabydosaur'
b'us\nacanthopholis\nache'


Organizando el conjunto para tener entradas y salidas donde ambos son secuencias. Por medio de la siguiente función se pretende obtener [input label]

In [18]:
def split_input_target(sequence):       #Recibe una secuencia como argumento
    input_text = sequence[:-1]          #Se toman los primeros n-1 caracteres de la secuencia
    target_text = sequence[1:]          #Se toman los caracteres del 1 al n de la secuencia
    return input_text, target_text

Ahora se aplica a todas las secuencias generadas, a traves del método map de python:

In [19]:
dataset = sequences.map(split_input_target)

Como ejemplo se muestra el resultado de [input, target] a partir de la separación explicada arriba:

In [20]:
for input_example, target_example in dataset.take(1):
    print("Input :", ind_a_texto(input_example).numpy())
    print("Target:", ind_a_texto(target_example).numpy())

Input : b'aachenosaurus\naardon'
Target: b'achenosaurus\naardony'


Antes de iniciar el entrenamiento se empaquetará el conjunto por lotes (batches) y revolver las secuencias:

In [21]:

BATCH_SIZE = 20               # Batch size

BUFFER_SIZE = 10000           #Para revolver (shuffle) las secuencias se recomienda hacerlo en un buffer de tamaño fijo para no ocupar completamente la memoria

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

print(dataset)

<_PrefetchDataset element_spec=(TensorSpec(shape=(20, 20), dtype=tf.int64, name=None), TensorSpec(shape=(20, 20), dtype=tf.int64, name=None))>


#### Construcción del modelo
A continuación se construye el modelo a partir de clases heredadas, el cual tendrá la siguiente estructura:



*   Capa embedding: Para evitar el one-hot-ncoding y aumentar innecesariamente la dimensionalidad del conjunto
*   Capa recurrente: Se realizarán modelos con rnn simple, LSTM y GRU
*   Capa densa: se encargará de ponderar y entregar el resultado de la genración de texto




Se inicializan los parámetros necesarios para las 3 capas:

In [22]:
# Longitud del vocabulario (caracteres únicos)
vocab_size = len(car_a_ind.get_vocabulary())
print(vocab_size)
# The embedding dimension
embedding_dim = 5

# Number of RNN units
lstm_units = 15

28


In [26]:
from keras.models import Sequential
inpu=tf.keras.Input(shape=(None,))            #Capa de entrada
model_sec = Sequential(name="First-LSTM-Model") # Creación de Modelo
#model.add(Input(shape=(time_step,1), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model_sec.add(tf.keras.Input(shape=(None,)))
model_sec.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
#if states is None:
  #model_sec.add(tf.keras.layers.LSTM(units=lstm_units,return_states=True, name='Hidden-Recurrent-Layer').get_initial_state()) # Hidden Recurrent Layer, Tanh(x) = sinh(x)/cosh(x) = ((exp(x) - exp(-x))/(exp(x) + exp(-x)))
model_sec.add(tf.keras.layers.LSTM(units=lstm_units, return_sequences=True, name='Hidden-Recurrent-Layer')) # Hidden Recurrent Layer, Tanh(x) = sinh(x)/cosh(x) = ((exp(x) - exp(-x))/(exp(x) + exp(-x)))
model_sec.add(Dense(units=vocab_size,  name='Hidden-Layer')) # Hidden Layer, Tanh(x) = sinh(x)/cosh(x) = ((exp(x) - exp(-x))/(exp(x) + exp(-x)))
#model.add(Dense(units=1, activation='linear', name='Output-Layer')) # Output Layer, Linear(x) = x
model_sec.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance.
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
             )
pred_train = model_sec(inpu)
model_sec.summary()

Model: "First-LSTM-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 5)           140       
                                                                 
 Hidden-Recurrent-Layer (LS  (None, None, 15)          1260      
 TM)                                                             
                                                                 
 Hidden-Layer (Dense)        (None, None, 28)          448       
                                                                 
Total params: 1848 (7.22 KB)
Trainable params: 1848 (7.22 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model_sec(input_example_batch)
    print("salida de modelo API secuencial",example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")



salida de modelo API secuencial (20, 20, 28) # (batch_size, sequence_length, vocab_size)


#### Realizando la primera predicción
A la salida se obtiene una distribución de probabilidad donde la salida con mayor probabilidad sera la neurona activa. Para mostrar la primera predicción sin entrenamiento, se tomará una de manera aleatoria y se organiza en un solo vector:

In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [29]:
sampled_indices

array([19, 19,  3, 24, 21, 27,  5, 22, 14, 10, 23, 18, 20, 25,  7, 17, 24,
        7,  2,  2])

Al decodificar el resultado obtenido:

In [30]:
print("Input:\n", ind_a_car(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", ind_a_texto(sampled_indices).numpy())

Input:
 [b'i' b'a' b'n' b's' b'a' b'u' b'r' b'u' b's' b'\n' b'e' b'r' b'l' b'i'
 b'k' b'o' b's' b'a' b'u' b'r']

Next Char Predictions:
 b'rrbwtzdumivqsxfpwfaa'


#### Entrenamiento del modelo
Despues de realizar todo el preprocesamiento del conjunto de datos el problema se puede tratar como un problema típico de clasificación

Se utilizara la función de pérdidas que usa la entropís cruzada que es bastante usada para los problemas de clasificación. Teniendo en cuenta que el modelo entrega valores de probabilidad (logits) se activa el argumento from_logits=True

In [31]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

Aún sin entrenar el modelo tiene unas pérdidas promedio de 3.33

In [32]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (20, 20, 28)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.3321848, shape=(), dtype=float32)


Teniendo en cuenta que el modelo está recién inicializado, al calcular e elevado a las pérdidas promedio se debería obtener un número cercano al tamaño del vocabulario. Si resulta mucho más grande esto indica que el modelo no está inicializado apropiadamente

In [33]:
tf.exp(example_batch_mean_loss).numpy()

27.999449

Ahora se compilará el modelo utilizando el optimizador Adam

In [34]:
model_sec.compile(optimizer='adam', loss=loss)
history_sec = model_sec.fit(dataset, epochs=400)

Epoch 1/400
47/47 [==============================] - 3s 22ms/step - loss: 3.2740
Epoch 2/400
47/47 [==============================] - 0s 4ms/step - loss: 3.0179
Epoch 3/400
47/47 [==============================] - 0s 4ms/step - loss: 2.8841
Epoch 4/400
47/47 [==============================] - 0s 7ms/step - loss: 2.8528
Epoch 5/400
47/47 [==============================] - 0s 4ms/step - loss: 2.8361
Epoch 6/400
47/47 [==============================] - 0s 4ms/step - loss: 2.8212
Epoch 7/400
47/47 [==============================] - 0s 4ms/step - loss: 2.8042
Epoch 8/400
47/47 [==============================] - 0s 4ms/step - loss: 2.7753
Epoch 9/400
47/47 [==============================] - 0s 4ms/step - loss: 2.7368
Epoch 10/400
47/47 [==============================] - 0s 7ms/step - loss: 2.6878
Epoch 11/400
47/47 [==============================] - 0s 4ms/step - loss: 2.6375
Epoch 12/400
47/47 [==============================] - 1s 8ms/step - loss: 2.5911
Epoch 13/400
47/47 [================

#### Generación de texto

La generación de texto se realizará corriendo el modelo en un loop. Cada vez que se llame el modelo recibirá texto y el estado anterior y la predicción será el siguiente caracter. En la siguiente iteración recibirá la predicción y el estado anterior para generar un nuevo caracter

In [35]:
next_char = tf.constant(['die'])


result = [next_char]
states=None

for n in range(50):
  input_chars = tf.strings.unicode_split(next_char, 'UTF-8')
  input_ids = car_a_ind(input_chars).to_tensor()
  print(input_chars)
  print(input_ids)

  predicted_logits = model_sec(inputs=input_ids)
  #print(predicted_logits)

  predicted_logits = predicted_logits[:, -1, :]
  #print(predicted_logits)
  predicted_logits = predicted_logits/1.0
  print(predicted_logits)

  predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
  #print(predicted_ids.numpy())
  predicted_ids = tf.squeeze(predicted_ids, axis=-1)
  print(predicted_ids.numpy())

  next_char = ind_a_car(predicted_ids)
  #print(next_char)
  result.append(next_char)
  #print(result)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

<tf.RaggedTensor [[b'd', b'i', b'e']]>
tf.Tensor([[ 5 10  6]], shape=(1, 3), dtype=int64)
tf.Tensor(
[[-17.423586    -1.6461207   -0.6249049   -1.8495392   -0.32624078
   -0.37992555  -1.368176    -4.2565722    1.3272929   -1.5641093
    0.15492988  -2.2532423   -2.226986     1.6032658   -0.3582132
    2.2883916    1.5658805    0.03693216  -4.1140623    0.41531602
    0.26914495   0.65691113   0.7185998   -0.2178269   -3.1853158
   -3.1254182   -0.6718692   -1.893971  ]], shape=(1, 28), dtype=float32)
[23]
<tf.RaggedTensor [[b'v']]>
tf.Tensor([[23]], shape=(1, 1), dtype=int64)
tf.Tensor(
[[-13.26162     -1.8370492    1.9753497   -2.275848    -3.1375637
   -2.0493405    3.3450794   -5.5925336   -4.442448    -0.2945557
    2.6150093   -5.78706     -1.7675182   -0.5993976   -3.146029
   -1.7811321    2.3561375   -0.7218998   -4.6728888    0.06532553
    0.26472253  -0.6707453    2.0587225   -3.9262128   -4.621283
   -4.3863044    0.91158724  -4.367721  ]], shape=(1, 28), dtype=float32)
[1